In [0]:
DEBUG = False

GALAXY = 'booi'
GALAXYFULLNAME = 'bootes'

CONFIG_FILE = f'../../../configs/netflow/dSph/{GALAXYFULLNAME}.py'

# Calibration target distribution

Plot the distribution of the calibration targets with cobra regions (sky and instrument) to verify that there are enough flux standards and sky positions to solve the netflow problem.

In [0]:
import os, sys
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

In [0]:
plt.rc('font', size=6)

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
if DEBUG and 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', int(os.environ['PFS_TARGETING_DEBUGPORT'])))
    debug = True

# Plot definitions

In [0]:
import pfs.utils
from pfs.ga.targeting.targets.dsph import *
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import WcsProjection, Pointing
from pfs.ga.targeting.netflow import Netflow
from pfs.ga.targeting.io import DataFrameSerializer

from astropy.time import Time, TimeDelta

In [0]:
galaxy = GALAXIES[GALAXY]
hsc = galaxy.get_photometry()
cmd = galaxy.get_cmd()
ccd = galaxy.get_ccd()
gaia_cmd = galaxy.get_cmd(Gaia)

In [0]:
pointings = galaxy.get_pointings(SubaruPFI)
pointing = pointings[0]

for p in pointings:
    print(p.ra, p.dec, p.posang)

In [0]:
wcs = WcsProjection(Pointing(galaxy.pos), proj='TAN')
wfc = SubaruWFC(pointing)
fov = FOV(projection=wcs)
fp = FP(wfc)

# Load the netflow config file

In [0]:
# Get the default configuration for the targets
config = galaxy.get_netflow_config()

# Load the customized configuration on top of the default one
config.load(CONFIG_FILE, ignore_collisions=True)

# Load the calibration target lists

In [0]:
from pfs.ga.targeting.io import ObservationSerializer
from pfs.ga.targeting.scripts.netflowscript import NetflowScript

from ics.cobraOps.TargetGroup import TargetGroup
from ics.cobraOps.TargetSelector import TargetSelector

In [0]:
target_lists = {}

for k in config.targets:

    if config.targets[k].prefix in ['sky', 'cal']:
        target_lists[k] = NetflowScript.load_target_list(config.targets[k])
        print(k, config.targets[k].prefix, config.targets[k].path)


In [0]:
config.instrument_options

In [0]:
pfi = SubaruPFI(instrument_options=config.instrument_options)

In [0]:
# Plot the targets within the field of view
for k in target_lists:
    f, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=240)
    fov.plot_catalog(ax, target_lists[k], size=0.5, color='lightgray')

    for p in pointings:
        pfi.plot_focal_plane(ax, fov, corners=True, projection=SubaruWFC(p))

    f.suptitle(f'{GALAXYFULLNAME} calibration targets {k}')

In [0]:
config.field.obs_time

In [0]:
def get_visible(pointing, target_list):
    # Calculate focal plane coordinates
    pointing.obs_time = Time(pointing.obs_time if pointing.obs_time is not None else config.field.obs_time)
    # print(pointing.ra, pointing.dec, pointing.obs_time)

    fp_pos = pfi.radec_to_fp_pos(pointing, *target_list.get_coords())
    fp_mask = np.abs(fp_pos) < 200
    # print(fp_pos.shape, (np.abs(fp_pos) < 200).sum())

    # Calculate the visibility of the targets
    class DummyTargetSelector(TargetSelector):
        def run(self):
            return

        def selectTargets(self):
            return
        
    tgroup = TargetGroup(fp_pos[fp_mask])
    tselect = DummyTargetSelector(pfi.bench, tgroup)
    tselect.calculateAccessibleTargets()
    targets = tselect.accessibleTargetIndices   # shape: (cobras, targets), padded with -1

    visible = defaultdict(list)

    for cidx in range(targets.shape[0]):
        for i, fpidx in enumerate(targets[cidx, :]):
            if fpidx >= 0:
                visible[cidx].append(fpidx)

    # Create a list of all visible targets
    visible_targets = []
    for cidx in visible:
        visible_targets.extend(visible[cidx])
    visible_targets = np.unique(np.array(visible_targets))

    return visible_targets, fp_pos, fp_mask, visible

In [0]:
def plot_visible(pointing, target_list, prefix, title):
    visible_targets, fp_pos, fp_mask, visible = get_visible(pointing, target_list)

    # print(targets.shape, visible_targets.shape)

    # Plot the visible targets on the focal plane
    f, axs = plt.subplots(1, 3, figsize=(12, 4), dpi=240)

    # Plot the visible targets on the focal plane
    fp.plot(axs[0], fp_pos[fp_mask].real[visible_targets], fp_pos[fp_mask].imag[visible_targets], ms=0.5, native_frame='pixel', alpha=0.5)
    pfi.plot_focal_plane(axs[0], fp, corners=True, projection=SubaruWFC(p))

    # Plot number of visible targets per cobra
    visible_count = {cidx: len(visible[cidx]) for cidx in visible}
    visible_count.update({ cidx: 0 for cidx in range(pfi.bench.cobras.nCobras) if cidx not in visible_count })
    hist = np.bincount(np.array([ v for v in visible_count.values() ]))
    axs[1].bar(np.arange(len(hist)), hist)
    axs[1].set_xlabel('Number of visible targets by cobra')
    axs[1].set_ylabel('Number of cobras')
    
    # Plot number of visible targets per cobra group
    for name, cobra_group in config.netflow_options.cobra_groups.items():
        if prefix in cobra_group.target_classes:
            # print(cobra_group.groups.min(), cobra_group.groups.max())

            # Sum up the number of visible targets per cobra group, ie. where the group identifier is the same
            group_visible = defaultdict(list)
            for cidx in visible_count:
                gidx = cobra_group.groups[cidx]
                group_visible[gidx].append(visible_count[cidx])

            group_visible_count = [ np.sum(group_visible[g]) for g in sorted(group_visible) ]
            # print(len(group_visible_count))

            axs[2].bar(np.arange(len(group_visible_count)), group_visible_count, label=name)

    axs[2].set_xlabel('Cobra group')
    axs[2].set_ylabel('Number of visible targets')
    axs[2].legend()

    f.suptitle(title)
    f.show()

In [0]:
for k in config.targets:
    for i, p in enumerate(config.pointings):
        print(p.ra, p.dec, p.posang, p.obs_time)
        if k in target_lists:
            plot_visible(p, target_lists[k], prefix=config.targets[k].prefix, title=f'Visible {k} targets for pointing {i}')